In [107]:
import itertools
from sys import maxsize
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import perfplot
from numba import jit, int64, types, typed, typeof, none, float32
import numba as nb
from numba.experimental import jitclass
from math import floor, sqrt, ceil
from time import time
from collections import namedtuple
np.random.seed(124)  # Seed the random number generator

#### Métodos quadráticos de ordenação

In [80]:
def insertion_sort(A):
    for j in range(1, len(A)):
        key = A[j]
        i = j - 1
        while (i >= 0) & (A[i] > key):
            A[i + 1] = A[i]
            i = i - 1
        A[i + 1] = key   
        
# def bubble_sort(V):
#     # Já que vamos comparar v[i] com v[i+1], no pior caso,
#     for iteração in range(len(V) - 1):
#         ocorreu_troca = False

#         for i in range((len(V) - 1) - iteração):
#             if V[i] > V[i + 1]:  # Troca os items
#                 V[i], V[i+ 1] = V[i+1], V[i]
#                 ocorreu_troca = True

#         if not ocorreu_troca:
#             # Se não ocorreu troca, quer dizer que ele já estava ordenado
#             break
#     return V



#### Square Root Sort

In [162]:
VAGO = -maxsize


@jitclass([("V", int64[:]), ("size", int64)])
class Parte(object):
    def __init__(self, V, size):
        self.V = V
        self.size = size


@jit(nopython=True)
def bubble_sort(parte: Parte):
    if parte.size > 1:
        for i in range(parte.size - 1):
            if parte.V[i] > parte.V[i + 1]:  # Troca os items
                parte.V[i], parte.V[i + 1] = parte.V[i + 1], parte.V[i]


@jit(nopython=True)
def particionar_array(array, qnt_partes, max_len_parte):
    n = len(array)

    partes = []

    for i in range(qnt_partes):
        if i != qnt_partes - 1:
            p = Parte(array[i * max_len_parte : (i + 1) * max_len_parte], max_len_parte)
        else:
            p = Parte(array[i * max_len_parte : n], n - (i * max_len_parte))

        partes.append(p)

    return partes


@jit(nopython=True)
def info_max_value(arr):
    max_value = -np.Infinity
    max_index_parte = 0

    for i, v in enumerate(arr):
        if (v != None) and (v > max_value):
            max_value = v
            max_index_parte = i

    return max_index_parte, max_value


@jit(nopython=True)
def sqrtsort(V):
    n = len(V)

    n_parte = floor(sqrt(n))
    n_partes = ceil(n / n_parte)

    partes = particionar_array(V, n_partes, n_parte)

    # aux_maiores = [None for _ in range(n_partes)]  # None == espaço vago
    aux_maiores = np.full(n_partes, VAGO)

    solucao = np.zeros(n, dtype=int64)

    for iteracao in range(n):
        # Preenche o vetor aux_maiores com os maiores valores de cada parte disponível
        for i, parte in enumerate(partes):

            # se está disponível e ainda existem elementos não ordenados
            if (aux_maiores[i] == VAGO) and (parte.size > 0):
                bubble_sort(parte)
                aux_maiores[i] = parte.V[parte.size - 1]

        # Pega o index e o valor do maior elemento de aux_maiores
        max_index, max_value = info_max_value(aux_maiores)

        solucao[iteracao] = max_value

        # Remove o maior elemento da parte correspondente
        parte = partes[max_index]

        if parte.size > 0:
            parte.size -= 1

        aux_maiores[max_index] = VAGO  # indica q a vaga está disponível

    return solucao[::-1]


runtimes = []
# sizes = np.linspace(10**2, 10**7, num=30, dtype=int)
# for size in sizes:
for _ in range(1):
    start = time()
    # array = np.random.randint(1000, size=size, dtype=np.int64)
    array = np.random.randint(1000, size=10**7, dtype=np.int64)

    res = sqrtsort(array)

    total = time() - start
    runtimes.append(total)
    print("{:.3f}".format(total), end="\r")

# plt.loglog(sizes, runtimes, label="Tempo")
# plt.show()
